In [1]:
import pandas as pd
import os
from pymongo import MongoClient
import unicodedata
import re

In [2]:
folder_path = "data/prf"
dfs = []

def split_marca_modelo(value):
    if isinstance(value, str) and '/' in value:
        return value.split('/', 1)  # Retorna uma lista com duas partes
    return [value, None]  # Retorna a marca e None para modelo se não houver barra

def normalize_string(s):
    # Remove acentos
    s = unicodedata.normalize('NFKD', s).encode('ASCII', 'ignore').decode('utf-8')
    # Transforma em minúsculas
    s = s.lower()
    # Substitui espaços e caracteres especiais por underscores
    s = re.sub(r'[^a-z0-9]+', '_', s)
    # Remove underscores do início e do fim
    s = s.strip('_')
    return s


def detect_delimiter(file_path):
    with open(file_path, 'r', encoding='ISO-8859-1') as file:
        first_line = file.readline()
        if ',' in first_line and ';' not in first_line:
            return ','
        elif ';' in first_line and ',' not in first_line:
            return ';'
        else:
            raise ValueError("Delimitador não identificado no arquivo: " + file_path)

for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    
    if file_name.endswith('.csv'):
        try:
            delimiter = detect_delimiter(file_path)
            new_df = pd.read_csv(file_path, delimiter=delimiter, encoding='ISO-8859-1', low_memory=False)

            new_df.replace("Não Informado", pd.NA, inplace=True)
            new_df.replace("Ignorado", pd.NA, inplace=True)
            new_df = new_df[new_df['ano_fabricacao_veiculo'] != 0]
            new_df = new_df[new_df['idade'] < 100]
            new_df = new_df[new_df['idade'] > 0]

            new_df[['marca_veiculo', 'modelo_veiculo']] = new_df['marca'].apply(split_marca_modelo).apply(pd.Series)

            new_df = new_df.dropna()

            col_to_normalize = ["dia_semana", "municipio", "causa_acidente", "marca",
                                "tipo_acidente", "classificacao_acidente", "fase_dia",
                                "sentido_via", "condicao_metereologica", "tipo_pista",
                                "tracado_via", "uso_solo", "tipo_veiculo", "tipo_envolvido",
                                "estado_fisico", "sexo", "regional", "delegacia", "uop", "nacionalidade", "naturalidade"
                                ]

            for coluna in col_to_normalize:
                if coluna in new_df.columns:
                    new_df[coluna] = new_df[coluna].apply(normalize_string)

            client = MongoClient('mongodb://localhost:27017/')
            db = client['data_science']
            collection = db['prf']

            documents = new_df.to_dict(orient='records')
            if documents:
                collection.insert_many(documents)

            print(f"{len(documents)} documentos inseridos com sucesso!")

            dfs.append(new_df)
        except Exception as e:
            print(f"Erro ao processar o arquivo {file_name}: {e}")

df = pd.concat(dfs, ignore_index=True)

173199 documentos inseridos com sucesso!
148935 documentos inseridos com sucesso!
98476 documentos inseridos com sucesso!
66849 documentos inseridos com sucesso!
176225 documentos inseridos com sucesso!
140210 documentos inseridos com sucesso!
137490 documentos inseridos com sucesso!
124404 documentos inseridos com sucesso!
126997 documentos inseridos com sucesso!
130747 documentos inseridos com sucesso!
141811 documentos inseridos com sucesso!
